In [ ]:
import nibabel as nib

# Replace 'path_to_input_image.nii.gz' with the path to your NIfTI image

img = nib.load('train/100101A/100101A_FLAIR.nii.gz')
data = img.get_fdata()
min_intensity = data.min()
max_intensity = data.max()

normalized_data = (data - min_intensity) / (max_intensity - min_intensity)
normalized_img = nib.Nifti1Image(normalized_data, img.affine)
nib.save(normalized_img, 'train-or/norm.nii.gz')



### Normailizing all the values


Performing image intensity normalization on NIfTI files involves scaling the pixel or voxel intensities to a common range, typically between 0 and 1 or -1 to 1. This process helps to make the data more comparable and suitable for further analysis or processing. Here's how you can perform image intensity normalization on NIfTI files using Python and the NiBabel library

In [ ]:
import os
import nibabel as nib

def normalize_nifti_files(input_folder, output_folder):
    # Get a list of all NIfTI files in the input folder and its subdirectories
    nifti_files = [file for file in os.listdir(input_folder) if file.endswith('.nii.gz')]

    for nifti_file in nifti_files:
        # Load the NIfTI image
        img = nib.load(os.path.join(input_folder, nifti_file))
        data = img.get_fdata()

        # Normalize the intensity
        min_intensity = data.min()
        max_intensity = data.max()
        normalized_data = (data - min_intensity) / (max_intensity - min_intensity)

        # Create the output folder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Save the normalized NIfTI image to the output folder
        output_file = os.path.join(output_folder, nifti_file)
        normalized_img = nib.Nifti1Image(normalized_data, img.affine)
        nib.save(normalized_img, output_file)

if __name__ == "__main__":
    input_folder = "train"  # Replace with the path to the folder containing patient subfolders
    output_folder = "train-or"  # Replace with the desired output folder for the normalized images

    for patient_folder in os.listdir(input_folder):
        # Process each patient folder and normalize the NIfTI files
        patient_input_folder = os.path.join(input_folder, patient_folder)
        patient_output_folder = os.path.join(output_folder, patient_folder)
        normalize_nifti_files(patient_input_folder, patient_output_folder)


### Noise reduction
- tried scipy but it increased the noise
- trying skimage is better and can see some improvment 

In [ ]:
import os
import nibabel as nib
from skimage.restoration import denoise_nl_means, estimate_sigma

def apply_noise_reduction(input_file, output_file, patch_size=3, patch_distance=1):
    # Load the NIfTI image
    img = nib.load(input_file)
    data = img.get_fdata()

    # Estimate the noise standard deviation from the image
    sigma_est = estimate_sigma(data)

    # Apply Non-Local Means denoising
    denoised_data = denoise_nl_means(data, patch_size=patch_size, patch_distance=patch_distance, h=sigma_est)

    # Save the denoised data to a new NIfTI file
    denoised_img = nib.Nifti1Image(denoised_data, img.affine)
    nib.save(denoised_img, output_file)

def process_patient_folder(input_folder, output_folder, patch_size=3, patch_distance=1):
    # Get a list of all NIfTI files in the input folder
    nifti_files = [file for file in os.listdir(input_folder) if file.endswith('.nii.gz')]

    for nifti_file in nifti_files:
        input_file = os.path.join(input_folder, nifti_file)
        output_file = os.path.join(output_folder, nifti_file)
        apply_noise_reduction(input_file, output_file, patch_size, patch_distance)

if __name__ == "__main__":
    input_folder = "train"  # Replace with the path to the folder containing patient subfolders
    output_folder = "train-or"  # Replace with the desired output folder for the denoised images
    patch_size = 3  # Adjust the patch size for denoising
    patch_distance = 1  # Adjust the patch distance for denoising

    for patient_folder in os.listdir(input_folder):
        # Process each patient folder and apply noise reduction
        patient_input_folder = os.path.join(input_folder, patient_folder)
        patient_output_folder = os.path.join(output_folder, patient_folder)

        # Create the patient-specific output folder if it doesn't exist
        if not os.path.exists(patient_output_folder):
            os.makedirs(patient_output_folder)

        # Process the patient folder and save denoised images
        process_patient_folder(patient_input_folder, patient_output_folder, patch_size, patch_distance)


### Bias Field correction?

In [2]:
import os
import nibabel as nib
import nipype.interfaces.ants as ants

def perform_bias_field_correction(input_file, output_file):
    # Initialize the N4BiasFieldCorrection algorithm
    n4 = ants.N4BiasFieldCorrection()

    # Set the input NIfTI image data and output filename for the corrected image
    n4.inputs.input_image = input_file
    n4.inputs.output_image = output_file

    # Run the bias field correction
    n4.run()

def process_patient_folder(input_folder, output_folder):
    # Get a list of all NIfTI files in the input folder
    nifti_files = [file for file in os.listdir(input_folder) if file.endswith('.nii.gz')]

    for nifti_file in nifti_files:
        input_file = os.path.join(input_folder, nifti_file)
        output_file = os.path.join(output_folder, nifti_file)

        # Perform bias field correction
        perform_bias_field_correction(input_file, output_file)

if __name__ == "__main__":
    input_folder = "train"  # Replace with the path to the folder containing patient subfolders
    output_folder = "train-or"  # Replace with the desired output folder for the corrected images

    for patient_folder in os.listdir(input_folder):
        # Process each patient folder and apply bias field correction
        patient_input_folder = os.path.join(input_folder, patient_folder)
        patient_output_folder = os.path.join(output_folder, patient_folder)

        # Create the patient-specific output folder if it doesn't exist
        if not os.path.exists(patient_output_folder):
            os.makedirs(patient_output_folder)

        # Process the patient folder and save corrected images
        process_patient_folder(patient_input_folder, patient_output_folder)


OSError: No command "N4BiasFieldCorrection" found on host charon-Legion-5-15ACH6. Please check that the corresponding package is installed.